<a href="https://colab.research.google.com/github/kata-n/python_google_colaboratory/blob/main/Websc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  ライブラリをインポート 
from bs4 import BeautifulSoup
import requests
url = "https://dempagumi.tokyo/"
r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")
tags = soup.find_all("a")
for tag in tags:
  print(tag.get("href"))

/
https://dempagumi.tokyo/news/
https://dempagumi.tokyo/schedule/
https://dempagumi.tokyo/profile/
https://dempagumi.tokyo/video/
https://dempagumi.tokyo/discography/
http://store.dearstage.com/shopbrand/dempa/
http://dempatow.dearstage.com/
https://twitter.com/dempagumi
https://ja-jp.facebook.com/dempagumi.inc/
https://lineblog.me/dempagumi/
https://dempagumi.tokyo/news/2020/12/25/210203/
https://dempagumi.tokyo/news/2020/11/11/nemopero_lavilith_1216event/
https://dempagumi.tokyo/news/2020/11/11/nemopero_lavilith_1216/
https://dempagumi.tokyo/news/2020/07/02/dempa_nemopero_jkt/
https://dempagumi.tokyo/news/2020/04/15/0415dempagumi_mv_info/
https://dempagumi.tokyo/news/2020/02/14/dempagumi_6thalbum/
https://dempagumi.tokyo/event/dempa_rh/
https://dempagumi.tokyo/event/aonyan_210411/
https://dempagumi.tokyo/schedule/
#all
#info
#live
#media
#goods
https://dempagumi.tokyo/news/2021/03/17/riapi_210404/
https://dempagumi.tokyo/news/2021/03/13/marche_210314/
https://dempagumi.tokyo/news/202